In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.io import loadmat
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import pickle

In [2]:
Categories= ["cyl","hook","lat","palm","spher","tip"]
pickle_in= open("Dicc.pickle","rb")
Data=pickle.load(pickle_in)
#Armarse los arrays
training_data=[]
def create_training_data():
    for category in Categories:
        DataCategory=Data[category]
        class_number=Categories.index(category)
        for i in range(len(DataCategory)):
            Arrays=DataCategory[i]
            training_data.append([Arrays,class_number])
                                 
create_training_data()
print(len(training_data))

2700


In [3]:
import random

random.shuffle(training_data)

In [5]:
X=[]
Y=[]

for data,label in training_data:
    X.append(data)
    Y.append(label)

X= np.array(X).reshape(-1,2500,2)
Y= np.array(Y).reshape(-1)

In [6]:


pickle_out= open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out= open("Y.pickle","wb")
pickle.dump(Y, pickle_out)
pickle_out.close()



In [7]:
#Convolutional Network
tf.keras.utils.normalize(X)

model=Sequential()
model.add(Conv1D(filters= 6, kernel_size= 7, input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3)) 

model.add(Conv1D(filters= 12, kernel_size= 7))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3)) 

model.add(Flatten(name = 'my_layer'))
model.add(Dense(200))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(6, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.fit(X,Y, batch_size=32, epochs=15, validation_split=0.3)

Train on 1889 samples, validate on 811 samples
Epoch 1/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 1.6549 - acc: 0.2927 - val_loss: 1.5228 - val_acc: 0.3477
Epoch 2/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 1.2791 - acc: 0.4960 - val_loss: 1.3985 - val_acc: 0.4192
Epoch 3/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 1.0592 - acc: 0.5696 - val_loss: 1.2902 - val_acc: 0.4846
Epoch 4/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 0.8472 - acc: 0.6617 - val_loss: 1.2412 - val_acc: 0.5351
Epoch 5/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 0.8013 - acc: 0.7073 - val_loss: 1.1295 - val_acc: 0.5697
Epoch 6/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 0.6527 - acc: 0.7470 - val_loss: 0.9555 - val_acc: 0.6313
Epoch 7/15
1889/1889 [==============================] - 2s 1ms/sample - loss: 0.5053 - acc: 0.8057 - val_loss: 1.0102 - val_acc: 0.6042
E

In [9]:
#Obtencion de salida de capa de caracteristicas
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X)
print(intermediate_output.shape)
pickle_out= open("output.pickle","wb")
pickle.dump(intermediate_output, pickle_out)
pickle_out.close()

(2700, 3300)
